# Peeker Options

Once the simulation has run and all the peekers have gathered their signal traces, you can show the waveforms in various ways.
The options that affect the waveform display will be illustrated using the hierarchical adder example shown below.

In [1]:
from myhdl import *
from myhdlpeek import Peeker, merge_all

def adder_bit(a, b, c_in, sum_, c_out):
    '''Single bit adder.'''
    @always_comb
    def adder_logic():
        sum_.next = a ^ b ^ c_in
        c_out.next = (a & b) | (a & c_in) | (b & c_in)
    
    # Add some peekers to monitor the inputs and outputs.
    p = Peeker(a=a, b=b, c_in=c_in, sum=sum_, c_out=c_out)
    
    return adder_logic

def adder(a, b, sum_):
    '''Connect single-bit adders to create a complete adder.'''
    c = [Signal(bool(0)) for _ in range(len(a)+1)] # Carry signals between stages.
    s = [Signal(bool(0)) for _ in range(len(a))] # Sum bit for each stage.
    stages = []  # Storage for adder bit instances.
    # Create the adder bits and connect them together.
    for i in range(len(a)):
        stages.append( adder_bit(a=a(i), b=b(i), sum_=s[i], c_in=c[i], c_out=c[i+1]) )
    # Concatenate the sum bits and send them out on the sum_ output.
    @always_comb
    def make_sum():
        sum_.next = ConcatSignal(*reversed(s))
    return instances()  # Return all the adder stage instances.

# Create signals for interfacing to the adder.
a, b, sum_ = [Signal(intbv(0,0,8)) for _ in range(3)]


# Instantiate the adder.
add_1 = adder(a=a, b=b, sum_=sum_)

# Create some more peekers to monitor the top-level buses.
p = Peeker(a_bus=a, b_bus=b, sum_bus=sum_, name='top')

# Create a testbench generator that applies random inputs to the adder.
from random import randrange
def test():
    for _ in range(8):
        a.next, b.next = randrange(0, a.max), randrange(0, a.max)
        yield delay(1)

ap = merge_all()
# Simulate the adder, testbench and peekers.
Simulation(add_1, test(), ap.instances(),).run()

<class 'myhdl.StopSimulation'>: No more events


0

## Selecting Waveforms to Display

By default, `to_wavedrom` shows all the captured waveforms. But you can also specify a subset of the waveforms for display:

In [3]:
p.to_wavedrom('a_bus', 'b_bus', 'sum_bus', 'sum[2]', 'sum[1]', 'sum[0]')

If you don't like typing all those quotation marks, you can place multiple, space-separated peeker names inside a string:

In [ ]:
Peeker.to_wavedrom('a_bus b_bus sum_bus sum[2] sum[1] sum[0]')

## Spacing the Waveforms

If you want to place some space between the waveforms, just insert a string that doesn't match any peeker's name (I like to use a `|` character for that):

In [ ]:
Peeker.to_wavedrom('a_bus b_bus | sum_bus sum[2] sum[1] sum[0]')

## Specifying a Time Window

To show only a segment of the waveforms, use the `start_time` and `stop_time` options:

In [ ]:
signals = 'a_bus b_bus | sum_bus sum[2] sum[1] sum[0]'
Peeker.to_wavedrom(signals, start_time=5, stop_time=15)

## Showing Cycle Times

If it's hard to tell the interval covered by the waveforms, you can turn on the display of cycle times using the `tick` or `tock` options:

In [ ]:
Peeker.to_wavedrom(signals, start_time=5, stop_time=15, tock=True)

## Adding Titles and Captions

You can also add a title and caption to your work:

In [ ]:
Peeker.to_wavedrom(signals, start_time=5, stop_time=15, tock=True,
                   title='Multi-Bit, Hierarchical Adder', caption='It really works!')

## Setting the Display Size

Possibly you have less screen real estate available. The `width` option lets you reduce the entire waveform display:

In [ ]:
Peeker.to_wavedrom(signals, start_time=5, stop_time=15, tock=True,
                   title='Multi-Bit, Hierarchical Adder', caption='It reall works!', width=400)

## Accessing the WaveJSON Data

Finally, you might want to get hold of the WaveJSON data directly to get more control over the waveform display. This is done with the `to_wavejson()` method:

In [ ]:
wavejson = Peeker.to_wavejson(signals)
wavejson

After you manipulate the WaveJSON data, you can display it using the `wavejson_to_wavedrom()` function:

In [ ]:
from myhdlpeek import wavejson_to_wavedrom
wavejson_to_wavedrom(wavejson)